# ML Pipeline Preparation_VK
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vitaliy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vitaliy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Vitaliy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vitaliy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# import libraries
import pandas as pd
import sqlite3
import re
import time
import numpy as np
import warnings
import pickle
warnings.simplefilter('ignore')


from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sqlalchemy import create_engine

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score, classification_report,make_scorer, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion

from scipy.stats import hmean


In [3]:
# load data from database
engine = create_engine('sqlite:///twitter_messages.db')
df = pd.read_sql('SELECT * FROM message',engine)

X = df['message']
y = df.iloc[:,4:40]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")

    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [11]:
def calculate_metrics(y_test, y_pred):
    res_metrics = []
    for i in range(y_pred.shape[1]):
        f1 = f1_score(y_test[:,i],y_pred[:,i])
        precision = precision_score(y_test[:,i],y_pred[:,i])
        recall = recall_score(y_test[:,i],y_pred[:,i])
        res_metrics.append([f1,precision,recall])
    return res_metrics

def median_f1(y_test,y_pred):
    f1 = []
    for i in range(y_pred.shape[1]):
        f1.append(f1_score(np.array(y_test)[:,i],y_pred[:,i]))
    return np.median(f1)

def f1_micro_average(y_test,y_pred):
    TN = []
    FP = []
    FN = []
    for i in range(y_pred.shape[1]):
        TN.append(confusion_matrix(np.array(y_test)[:,i],y_pred[:,i])[1,1])
        FP.append(confusion_matrix(np.array(y_test)[:,i],y_pred[:,i])[1,0])
        FN.append(confusion_matrix(np.array(y_test)[:,i],y_pred[:,i])[0,1])
    precision = np.sum(TN) / (np.sum(TN) + np.sum(FN))
    recall = np.sum(TN) / (np.sum(TN) + np.sum(FP))
    
    return hmean([precision,recall])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 1, test_size = 0.2)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
start = time.time()
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)
print('F1_Median: {}'.format(median_f1(y_test,y_pred)))
print('F1_micro_average: {}'.format(f1_micro_average(y_test,y_pred)))
print('Time required: {}'.format(time.time()-start))

F1_Median: 0.14492753623188404
F1_micro_average: 0.6152200488997556
Time required: 226.63196277618408


In [ ]:
test_res = calculate_metrics(y_test.values,y_pred)
print('Result on Test Data using untuned MultiOutputClassifier with Random Forest')
print(pd.DataFrame(test_res, columns = ['f1','precision','recall'], index = df.iloc[:,4:40].columns ) )

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Create grid search object

parameters = {'vect__min_df': [1, 5],
              'clf__estimator__n_estimators': [50, 100],
             'clf__estimator__base_estimator__min_samples_split':[2,50,500],
             'clf__estimator__base_estimator__max_depth':[1,5]}

scorer = make_scorer(f1_micro_average)

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10, cv = 2)

tuned_model = cv.fit(X_train, y_train)

In [ ]:
# Create grid search object

parameters = {'vect__min_df': [1, 10],
              'clf__estimator__n_estimators': [10, 50], 
              'clf__estimator__learning_rate':[0.1,1,10]}

scorer = make_scorer(f1_micro_average)

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10, cv = 3)

# Find best parameters

tuned_model = cv.fit(X_train, y_train)

In [ ]:
tuned_model.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(tuned_model, open('try.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
loaded_model = pickle.load(open('try.sav', 'rb'))